In [15]:
from pybliometrics.scopus import ScopusSearch
from pybliometrics.scopus.utils import config
import pandas as pd

best_papers = pd.read_csv("./data/best_scopus.csv")
best_papers['best_yn'] = 1
scopus_query = pd.read_csv("./data/best_scopus_whole_madequery.csv")
scopus_query['best_yn'] = 0
scopus_load = pd.read_csv("./data/best_scopus_whole.csv")
scopus_load['best_yn'] = 0

/home/pkmon1d/miniconda3/lib/python3.11/site-packages/pandas/io/parsers/c_parser_wrapper.py:365: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common_type = np.find_common_type(
/home/pkmon1d/miniconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (2,19,24,25) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/pkmon1d/miniconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (19,21,24,25) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [73]:
#whole_data = pd.concat([best_papers, scopus_load])
whole_data = pd.concat([best_papers, scopus_query])
whole_data = pd.concat([whole_data, scopus_load])
whole_data = whole_data.drop_duplicates(subset='eid')
whole_data['year'] = whole_data['coverDate'].str[:4]
whole_data = whole_data.loc[(whole_data['affiliation_country'].notna())& (whole_data['year'] >= '1996')]

/home/pkmon1d/miniconda3/lib/python3.11/site-packages/pandas/core/dtypes/cast.py:1846: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])  # type: ignore[arg-type]


In [74]:
whole_data.groupby('origin_ref')['eid'].count().reset_index().sort_values('origin_ref')

,origin_ref,eid
0,AAAI Artificial Intelligence,18167
1,ACL Natural Language Processing,13547
2,CHI Human-Computer Interaction,19988
3,CIKM Knowledge Management,10527
4,CVPR Computer Vision,17351
5,FOCS Theory,4377
6,FSE Software Engineering,2326
7,ICCV Computer Vision,20809
8,ICML Machine Learning,10877
9,ICSE Software Engineering,30065


In [75]:
whole_data.groupby('year')['eid'].count().reset_index().sort_values('year')

,year,eid
0,1996,2191
1,1997,2603
2,1998,2907
3,1999,2933
4,2000,2711
5,2001,3294
6,2002,2835
7,2003,4317
8,2004,4537
9,2005,5995


In [76]:
whole_data['author_count'] = whole_data['author_count'].fillna(0).astype(int)
print("논문 별 평균 저자 수",whole_data['author_count'].mean())

df_exploded = whole_data.assign(affiliation_country=whole_data['affiliation_country'].str.split(';')).explode('affiliation_country')
print("논문 별 평균 공동 작업 국가 수", df_exploded.groupby('eid')['affiliation_country'].nunique().mean())

논문 별 평균 저자 수 3.677997076023392
논문 별 평균 공동 작업 국가 수 1.353431980624963


In [77]:
###공동작업 국가수, Paper수
#Affiliation 정보가 없는 경우, 공동작업 국가 수가 0
df_exploded.groupby('eid')['affiliation_country'].nunique().reset_index().groupby('affiliation_country').count().reset_index().head(10)

,affiliation_country,eid
0,1,195504
1,2,59370
2,3,12876
3,4,2395
4,5,490
5,6,127
6,7,45
7,8,23
8,9,8
9,10,7


In [78]:
###국가별 저자 수
df_exploded.groupby("affiliation_country")['eid'].count().reset_index().sort_values('eid', ascending= False).head(10)

,affiliation_country,eid
146,United States,183991
28,China,76811
50,Germany,25857
145,United Kingdom,25407
25,Canada,17557
48,France,17354
69,Japan,12775
6,Australia,12073
58,Hong Kong,9153
67,Italy,8972


In [79]:
df_exploded = whole_data.assign(affilname=whole_data['affilname'].str.split(';')).explode('affilname')
df_exploded.groupby("affilname")['eid'].count().reset_index().sort_values('eid', ascending= False).head(10)

,affilname,eid
14996,Microsoft Research,6120
3673,Carnegie Mellon University,5163
23162,Tsinghua University,4946
8586,Google LLC,4627
24518,"University of California, Berkeley",4452
6051,Department of Computer Science,4319
21309,Stanford University,3733
6659,ETH Zürich,3187
14496,Massachusetts Institute of Technology,3164
25101,University of Washington,3052


In [62]:
mini_data = whole_data.loc[whole_data['best_yn']==1]
mini_data.groupby('origin_ref')['eid'].count().reset_index().sort_values('origin_ref')

,origin_ref,eid
0,AAAI Artificial Intelligence,43
1,ACL Natural Language Processing,35
2,CHI Human-Computer Interaction,325
3,CIKM Knowledge Management,22
4,CVPR Computer Vision,26
5,FOCS Theory,39
6,FSE Software Engineering,103
7,ICCV Computer Vision,20
8,ICML Machine Learning,41
9,ICSE Software Engineering,142


In [64]:
mini_data['author_count'] = mini_data['author_count'].fillna(0).astype(int)
print("논문 별 평균 저자 수",mini_data['author_count'].mean())

df_exploded = mini_data.assign(affiliation_country=mini_data['affiliation_country'].str.split(';')).explode('affiliation_country')
print("논문 별 평균 공동 작업 국가 수", df_exploded.groupby('eid')['affiliation_country'].nunique().mean())

논문 별 평균 저자 수 4.12697323266987
논문 별 평균 공동 작업 국가 수 1.3843514070006864


/tmp/ipykernel_847786/1665527498.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mini_data['author_count'] = mini_data['author_count'].fillna(0).astype(int)


In [65]:
###국가별 저자 수
df_exploded.groupby("affiliation_country")['eid'].count().reset_index().sort_values('eid', ascending= False).head(10)

,affiliation_country,eid
48,United States,1482
47,United Kingdom,200
8,China,148
6,Canada,143
14,Germany,140
13,France,77
43,Switzerland,68
1,Australia,59
21,Israel,49
27,Netherlands,41


In [66]:
df_exploded = mini_data.assign(affilname=mini_data['affilname'].str.split(';')).explode('affilname')
df_exploded.groupby("affilname")['eid'].count().reset_index().sort_values('eid', ascending= False).head(10)

,affilname,eid
356,Microsoft Research,95
744,University of Washington,74
81,Carnegie Mellon University,62
665,"University of California, Berkeley",54
200,Google LLC,44
527,Stanford University,43
707,"University of Michigan, Ann Arbor",39
737,University of Toronto,31
340,Massachusetts Institute of Technology,31
486,School of Computer Science,27


In [50]:
high_cited_data = whole_data.loc[whole_data['citedby_count']>=15]
len(high_cited_data)

107332

In [51]:
high_cited_data['author_count'] = high_cited_data['author_count'].fillna(0).astype(int)
print("논문 별 평균 저자 수",high_cited_data['author_count'].mean())

df_exploded = high_cited_data.assign(affiliation_country=high_cited_data['affiliation_country'].str.split(';')).explode('affiliation_country')
print("논문 별 평균 공동 작업 국가 수", df_exploded.groupby('eid')['affiliation_country'].nunique().mean())

/tmp/ipykernel_847786/3731487723.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_cited_data['author_count'] = high_cited_data['author_count'].fillna(0).astype(int)


논문 별 평균 저자 수 3.8666753624268626
논문 별 평균 공동 작업 국가 수 1.3931073677933887


In [52]:
###국가별 저자 수
df_exploded.groupby("affiliation_country")['eid'].count().reset_index().sort_values('eid', ascending= False).head(20)

,affiliation_country,eid
106,United States,90555
19,China,27772
105,United Kingdom,10624
34,Germany,9481
16,Canada,7477
33,France,5963
6,Australia,5059
37,Hong Kong,4502
96,Switzerland,3765
45,Israel,3740


In [36]:
whole_data.to_csv('./data/ai_research_papers.csv', index= False)